In [35]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [36]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import (LLMChain, SequentialChain)


In [38]:
llm = ChatOpenAI(temperature=0.0)


## Chain 1

Determine the Language of the Review

In [39]:
first_prompt = ChatPromptTemplate.from_template(
    "Determine and return only the langauge of:\n\n{Review}, i.e. English, French, Spanish etc."
)
# chain 3: input= Review and output= language
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                       output_key="language"
                      )

## Chain 2

Translate reviews to English if not in English

In [40]:
second_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english only if {language} is not English, otherwise just leave it as is:"
    "\n\n{Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="English_Review"
                    )

## Chain 3

Summarize the review to a single sentence

In [41]:
third_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_three = LLMChain(llm=llm, prompt=third_prompt, 
                     output_key="summary"
                    )

## Chain 4

Determine the sentiment of the review as either Positive or Negative

In [42]:
fourth_prompt = ChatPromptTemplate.from_template(
    "What is the sentiment of the following review, only answer as Positive or Negative:\n\n{Review}"
)

chain_four = LLMChain(llm =llm, prompt=fourth_prompt,
                      output_key="sentiment")

## Chain 5

Give the review a score between 0 and 5

In [43]:
fifth_prompt = ChatPromptTemplate.from_template(
    "Determine the score between 0 to 5 for following review, where 0 is very negative and 5 is very positive, only provide the score:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_five = LLMChain(llm=llm, prompt=fifth_prompt,
                       output_key="review_score"
                      )

In [44]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four , chain_five],
    input_variables=["Review"],
    output_variables=["English_Review", "language" , "summary", "sentiment" , "review_score"],
    verbose=True
)

In [45]:
review = df.Review[1]
print(review)
msg = overall_chain(review)
print(msg)

I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.


> Entering new  chain...

> Finished chain.
{'Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.', 'English_Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.', 'language': 'The language of the given text is English.', 'summary': 'The reviewer enjoyed the waterproof sac but had concerns about the durability of the hard plastic opening and experienced difficulty turning on their phone while it was inside the pouch.', 'sentiment': 'Negative', 'review_score': '3'}


In [34]:
print(msg)

{'Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.', 'English_Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.', 'language': 'The language of the review is English.', 'summary': 'The reviewer enjoyed the waterproof sac but had concerns about the durability of the hard plastic opening and experienced difficulty turning on their phone while it was inside the pouch.', 'sentiment': 'Negative', 'review_score': '3'}


In [15]:
resp = df['Review'].apply(lambda x:overall_chain(x))



> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


In [19]:
resp

df3 = pd.DataFrame.from_records(resp)

df3

,Review,English_Review,language,summary,sentiment,review_score
0,I ordered a king size set. My only criticism w...,I ordered a king size set. My only criticism w...,The language of the review is English.,The reviewer ordered a king size set of sheets...,Positive,4
1,"I loved the waterproof sac, although the openi...","I loved the waterproof sac, although the openi...",The language of the review is English.,The reviewer enjoyed the waterproof sac but ha...,Negative,3
2,This mattress had a small hole in the top of i...,This mattress had a small hole in the top of i...,The language of the review is English.,The reviewer had a terrible experience with a ...,Negative,Score: 0
3,This is the best throw pillow fillers on Amazo...,This is the best throw pillow fillers on Amazo...,The language of the review is English.,The reviewer highly recommends these throw pil...,Positive,4.5
4,I loved this product. But they only seem to l...,I loved this product. But they only seem to la...,The language of the review is English.,"The reviewer loved the product initially, but ...",Negative,2
5,Je trouve le goût médiocre. La mousse ne tient...,I find the taste mediocre. The foam doesn't ho...,The language of the review is French.,The reviewer is disappointed with the taste an...,Negative,2
6,"Está lu bonita calienta muy rápido, es muy fun...","It is a very nice heater, it heats up very qui...",The language of the review is Spanish.,The reviewer finds the heater to be efficient ...,Positive,4


In [46]:
responses = []

for i in range(0,5):
   review = df.Review[i]
   print(review)
   msg = overall_chain(review)
   responses.append(msg)
   

I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.


> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.
I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.


> Entering new  chain...

> Finished chain.
This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.


> Entering new  chain...

> Finished chain.
This is the best thro

In [18]:
df3 = pd.DataFrame.from_records(resp)

df3

,Review,English_Review,language,summary,sentiment,review_score
0,I ordered a king size set. My only criticism w...,I ordered a king size set. My only criticism w...,The language of the review is English.,The reviewer ordered a king size set of sheets...,Positive,4
1,"I loved the waterproof sac, although the openi...","I loved the waterproof sac, although the openi...",The language of the review is English.,The reviewer enjoyed the waterproof sac but ha...,Negative,3
2,This mattress had a small hole in the top of i...,This mattress had a small hole in the top of i...,The language of the review is English.,The reviewer had a terrible experience with a ...,Negative,Score: 0
3,This is the best throw pillow fillers on Amazo...,This is the best throw pillow fillers on Amazo...,The language of the review is English.,The reviewer highly recommends these throw pil...,Positive,4.5
4,I loved this product. But they only seem to l...,I loved this product. But they only seem to la...,The language of the review is English.,"The reviewer loved the product initially, but ...",Negative,2
5,Je trouve le goût médiocre. La mousse ne tient...,I find the taste mediocre. The foam doesn't ho...,The language of the review is French.,The reviewer is disappointed with the taste an...,Negative,2
6,"Está lu bonita calienta muy rápido, es muy fun...","It is a very nice heater, it heats up very qui...",The language of the review is Spanish.,The reviewer finds the heater to be efficient ...,Positive,4


In [48]:
for i in range(0,5): print(responses[i] , end ="\n")

{'Review': 'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.', 'English_Review': 'I ordered a king size set. My only criticism would be that I wish the seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and c

In [71]:
df.merge(df3)

,Product,Review,English_Review,language,summary,sentiment,review_score
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...,I ordered a king size set. My only criticism w...,The following review is in English.,The reviewer ordered a king size set of sheets...,Positive,"Based on the review, I would give a score of 4."
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi...","I loved the waterproof sac, although the openi...",The language of the review appears to be English.,The reviewer enjoyed the waterproof sac but ha...,Negative,"Based on the review, I would assign a score of..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...,This mattress had a small hole in the top of i...,The following review is in English.,The reviewer had a terrible experience with a ...,Negative,"Based on the review, the score would be 0."
3,Pillows Insert,This is the best throw pillow fillers on Amazo...,This is the best throw pillow fillers on Amazo...,The following review is in English.,The reviewer highly recommends these throw pil...,Positive,4.5
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...,I loved this product. But they only seem to la...,The following review is in English.,"The reviewer loved the product initially, but ...",Negative,"Based on the review, I would assign a score of..."
